## Querying relational database Country Club on Springboard directly with pandas

In [15]:
from sqlalchemy import create_engine
import pandas as pd
# engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
engine = create_engine('sqlite:///C:/Sereda/Lectures/Springboard/8.3.3 SQL - Country Club/SQLFiles Tier 2/sqlite_db_pythonsqlite.db')

In [16]:
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


### Test data import

In [19]:
df = pd.read_sql_query("SELECT * FROM Facilities LIMIT 5", engine)
df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


# QUESTIONS:
### Q10
Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [21]:
df = pd.read_sql_query("SELECT s.name AS facility, s.total_revenue  FROM (SELECT f.name, SUM( CASE WHEN (m.firstname != 'GUEST')  THEN f.membercost * b.slots  ELSE f.guestcost * b.slots  END) AS total_revenue  FROM Bookings AS b  LEFT JOIN Facilities AS f  ON b.facid = f.facid  LEFT JOIN Members AS m  ON b.memid = m.memid  GROUP BY f.facid) AS s  WHERE s.total_revenue < 1000;", engine)
df.head()

,facility,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


### Q11
Produce a report of members and who recommended them in alphabetic surname, firstname order

In [29]:
df = pd.read_sql_query("SELECT DISTINCT CASE WHEN (m.surname != 'GUEST') THEN (m.surname || ' ' || m.firstname)  ELSE 'GUEST'  END AS member,  CASE WHEN (m.recommendedby = '') THEN ''  ELSE (m2.surname || ' ' || m2.firstname)  END AS recommender  FROM Members AS m  LEFT JOIN Members AS m2  ON m.recommendedby = m2.memid  ORDER BY member;", engine)
df

,member,recommender
0,Bader Florence,Stibbons Ponder
1,Baker Anne,Stibbons Ponder
2,Baker Timothy,Farrell Jemima
3,Boothe Tim,Rownam Tim
4,Butters Gerald,Smith Darren
5,Coplin Joan,Baker Timothy
6,Crumpet Erica,Smith Tracy
7,Dare Nancy,Joplette Janice
8,Farrell David,
9,Farrell Jemima,


Note: use '||' for concatenation.

### Q12
Find the facilities with their usage by member, but not guests

In [33]:
df = pd.read_sql_query("SELECT f.name AS facility,  b.memid,  SUM(b.slots) AS usag  FROM Bookings AS b  LEFT JOIN Facilities AS f  ON b.facid = f.facid  WHERE b.memid != 0  GROUP BY f.facid, b.memid  ORDER BY f.facid, b.memid;", engine)
df # List of facilities used by members, with details on usage by member ID

,facility,memid,usag
0,Tennis Court 1,2,93
1,Tennis Court 1,3,18
2,Tennis Court 1,4,57
3,Tennis Court 1,5,171
4,Tennis Court 1,6,93
...,...,...,...
197,Pool Table,27,3
198,Pool Table,28,25
199,Pool Table,29,37
200,Pool Table,30,5


In [32]:
df = pd.read_sql_query("SELECT f.name AS facility,  CASE WHEN (m.surname != 'GUEST') THEN  (m.surname || ' ' || m.firstname)  ELSE 'GUEST'  END AS member,  SUM(b.slots) AS usag  FROM Bookings AS b  LEFT JOIN Facilities AS f  ON b.facid = f.facid  LEFT JOIN Members AS m  ON m.memid = b.memid  WHERE b.memid != 0  GROUP BY f.facid, b.memid  ORDER BY f.facid, member;", engine)
df # List of facilities used by members, with details on usage by member name

,facility,member,usag
0,Tennis Court 1,Bader Florence,3
1,Tennis Court 1,Baker Anne,18
2,Tennis Court 1,Baker Timothy,45
3,Tennis Court 1,Boothe Tim,12
4,Tennis Court 1,Butters Gerald,171
...,...,...,...
197,Pool Table,Smith Tracy,64
198,Pool Table,Stibbons Ponder,12
199,Pool Table,Tracy Burton,30
200,Pool Table,Tupperware Hyacinth,9


### Q13
Find the facilities usage by month, but not guests

In [38]:
df = pd.read_sql_query("SELECT f.name AS facility,  strftime ('%m', b.starttime) AS month,  SUM(b.slots) AS usag  FROM Bookings AS b  LEFT JOIN Facilities AS f  ON b.facid = f.facid  WHERE b.memid != 0  GROUP BY f.facid, month  ORDER BY f.facid, month;", engine)
df

,facility,month,usag
0,Tennis Court 1,07,201
1,Tennis Court 1,08,339
2,Tennis Court 1,09,417
3,Tennis Court 2,07,123
4,Tennis Court 2,08,345
5,Tennis Court 2,09,414
6,Badminton Court,07,165
7,Badminton Court,08,414
8,Badminton Court,09,507
9,Table Tennis,07,98


In [ ]:
Note: SQLite does not support EXTRACT (), MONTH () or DATEPART () function.